In [1]:
from tpot import TPOTClassifier, TPOTRegressor
import dill
import pandas as pd

from configparser import ConfigParser
import numerapi
import os
import sys
if not os.getcwd().endswith('trading'): os.chdir('../../../trading') # local machine
assert os.getcwd().endswith('trading'), 'Wrong path!'
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from collections import Counter
from skimpy import clean_columns

from sklearn.metrics import mean_squared_error
from pandarallel import pandarallel # parallel pandas
import platform
import time
import numpy as np
sys.path.append(os.getcwd())

In [2]:
model_dict = dill.load(open('/media/melgazar9/HDD_10TB/trading/objects/XGBRegressor_2022-02-28_13_33_00.pkl', 'rb'))

/home/melgazar9/scripts/github/trading/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### train test split

In [12]:
TARGET = 'target_20d'
PRESERVE_VARS = ['XGBRegressor_pred', 'target_4d', 'yahoo_ticker', 'date_localized', 'dataset_split']
ID_VARS = ['date']

DROP_COLS = model_dict['dropped_features'] # [i for i in model_dict['df_pred'].columns if is_datetime(model_dict['df_pred'][i]) and i not in ID_VARS + PRESERVE_VARS]
OBJECT_COLS = [i for i in model_dict['df_pred'].columns if model_dict['df_pred'][i].dtype == 'object' and i not in ID_VARS + PRESERVE_VARS]

In [13]:
try:
    model_dict['df_pred'].set_index(ID_VARS, inplace=True)
except:
    pass
try:
    model_dict['df_pred'].drop(DROP_COLS, axis=1, inplace=True)
except:
    pass

In [23]:
X_train = model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='train'][model_dict['input_features']]
y_train = model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='train'][TARGET]

X_val = model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='val'][model_dict['input_features']]
y_val = model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='val'][TARGET]

X_test = model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='test'][model_dict['input_features']]
y_test = model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='test'][TARGET]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((939046, 813), (939046,), (201224, 813), (201224,), (201225, 813), (201225,))

In [24]:
X_train.head()

,volume_1h_5,high_minus_prev_close_1h_4,move_pct_1h_9,move_pct_change_1d_ewm_mean,low_move_pct_1h_5_rolling_mean,high_move_1h_2_rolling_mean,low_1h_5_diff,high_1h_6_diff,high_move_1h_1_ewm_mean,low_minus_close_1h_5,...,move_pct_change_1h_2_ewm_mean,low_move_pct_1h_11_ewm_mean,high_move_pct_1h_13_ewm_mean,high_move_1d_rolling_mean,high_move_pct_1h_11,adj_close_1h_8_diff,low_move_1h_3_ewm_mean,high_move_1h_0_rolling_mean,volume_pct_change_1h_3,move_pct_change_1h_5_ewm_mean
date,,,,,,,,,,,,,,,,,,,,,
2015-02-06,NaN,NaN,NaN,-0.650000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-13,NaN,NaN,NaN,-0.091873,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-27,NaN,NaN,NaN,-9.390316,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-06,NaN,NaN,NaN,-3.426383,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-13,NaN,NaN,NaN,-0.954995,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
X_train_transformed = pd.DataFrame(model_dict['feature_transformer'].transform(X_train), columns=model_dict['final_features'])
X_val_transformed = pd.DataFrame(model_dict['feature_transformer'].transform(X_val), columns=model_dict['final_features'])
X_test_transformed = pd.DataFrame(model_dict['feature_transformer'].transform(X_test), columns=model_dict['final_features'])

### Original XGBRegressor score

In [27]:
print('\nTrain xgboost RMSE: {}\n'.format(
    np.sqrt(mean_squared_error(model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='train']['XGBRegressor_pred'],
                       model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='train']['target_20d']))))

print('Train xgboost RMSE: {}\n'.format(
    np.sqrt(mean_squared_error(model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='val']['XGBRegressor_pred'],
                       model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='val']['target_20d']))))

print('Train xgboost RMSE: {}\n'.format(
    np.sqrt(mean_squared_error(model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='test']['XGBRegressor_pred'],
                       model_dict['df_pred'][model_dict['df_pred']['dataset_split']=='test']['target_20d']))))



Train xgboost RMSE: 0.2122374027967453

Train xgboost RMSE: 0.22152180969715118

Train xgboost RMSE: 0.216725155711174



In [41]:
from sklearn.preprocessing import FunctionTransformer, OrdinalEncoder, LabelEncoder

In [43]:
tmp = LabelEncoder().fit_transform(X_train['bloomberg_ticker'])

In [44]:
tmp

array([   0,    0,    0, ..., 3595, 3595, 3595])

In [11]:
tpot_model = TPOTRegressor(generations=5,
                           population_size=5,
                           offspring_size=None,
                           mutation_rate=0.9,
                           scoring='neg_mean_squared_error',
                           cv=3,
                           subsample=0.8,
                           n_jobs=16,
                           random_state=0,
                           use_dask=True,
                           early_stop=5).fit(X_train, y_train)

ValueError: Cannot cast object dtype to float64